On 1-Dec-2020, the S&P500 (SPX) index value was 3662.45, while the SPDR
S&P500 Exchange Traded Fund (SPY) stock price was 366.02. The call and
put option prices (bid & offer) over 3 maturities are provided in the
spreadsheet:
• SPX options.csv
• SPY options.csv
The discount rate on this day is in the file: zero rates 20201201.csv.
Calibrate the following models to match the option prices:
1 Displaced-diffusion model
2 SABR model (fix β = 0.7)
Plot the fitted implied volatility smile against the market data.
Report the model parameters:
1 σ, β
2 α, ρ, ν
And discuss how does change β in the displaced-diffusion model and ρ, ν in the
SABR model affect the shape of the implied volatility

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq, least_squares
import matplotlib.pylab as plt

First, we shall implement the SABR model. The implied Black volatility of the SABR model is given by

\begin{equation}
\sigma_{\text{SABR}}(F_0, K, \alpha, \beta, \rho, \nu) = \frac{\alpha}{(F_0K)^{(1-\beta)/2} \left\{ 1 + \frac{(1-\beta)^2}{24}\log^2\left(\frac{F_0}{K}\right) + \frac{(1-\beta)^4}{1920}\log^4\left(\frac{F_0}{K}\right) + \cdots \right\} } 
\cdot \frac{z}{x(z)} \cdot \left\{ 1 + \left[ \frac{(1-\beta)^2}{24} \frac{\alpha^2}{(F_0K)^{1-\beta}} + \frac{1}{4} \frac{\rho \beta \nu \alpha}{(F_0K)^{(1-\beta)/2}} + \frac{2-3\rho^2}{24}\nu^2 \right] T + \cdots \right\},
\notag
\end{equation}

where

\begin{equation}
z = \frac{\nu}{\alpha} (F_0K)^{(1-\beta)/2} \log\left(\frac{F_0}{K}\right), \notag
\end{equation}

and

\begin{equation}
x(z) = \log \left[ \frac{\sqrt{1-2\rho z+z^2} + z - \rho}{1 - \rho} \right]. \notag
\end{equation}


In [32]:
def BSimpliedVolatility(S, K, r, price, T, payoff):
    """
    Function to approximate implied volatility based on given option price info using BS model
    """
    try:
        if payoff:
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalCall(S, K, r, x, T),
                                -1e-12, 10.0)
        else:
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalPut(S, K, r, x, T),
                                1e-12, 10.0)   
    except Exception:
        impliedVol = np.nan
    
    return impliedVol


def BlackScholesLognormalCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesLognormalPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

In [33]:
# my version

df = pd.read_csv("SPX_options.csv")
df['mid'] = .5*(df['best_bid'] + df['best_offer'])
df['strike'] = df['strike_price']*.001                                # strike price is in a different scale
df['call_flag'] = df['cp_flag'].map(lambda x: 1 if x == 'C' else 0)

# choose exercise date and get days to expiry
exdate = sorted(df['exdate'].unique())[0]
df = df[df["exdate"] == exdate]
days_to_expiry = (pd.Timestamp(str(exdate)) - pd.Timestamp('2020-12-01')).days    

# set parameters for option of above ex date
T = days_to_expiry/365
S = 3662.45
r = 0.14/100.0          # TODO need to change based on zero curve
F = S*np.exp(r*T)

# get list of implied vols based on black-scholes model
df['vols'] = df.apply(lambda x: BSimpliedVolatility(S,
                                                    x['strike'],
                                                    r,
                                                    x['mid'],
                                                    T,
                                                    x['call_flag']),
                                                    axis=1)

# get all OTM call and put strikes and combine to one dataframe
df.dropna(inplace=True)             # drop NA due to brentq not able to find vols for some calls
call_df = df[df['call_flag'] == 1]
put_df = df[df['call_flag'] == 0]
strikes = put_df['strike'].values
impliedvols = []
for K in strikes:
    if K>S:
        impliedvols.append(call_df[call_df['strike'] == K]['vols'].values[0])
    else:
        impliedvols.append(put_df[put_df['strike'] == K]['vols'].values[0])
df_iv = pd.DataFrame({'strike': strikes, 
                      'impliedvol': impliedvols})



In [37]:
call_df[(df['strike'] >= 2900) & (df['strike'] <= 3100)]

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_97651/3406394907.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  call_df[(df['strike'] >= 2900) & (df['strike'] <= 3100)]


,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid,strike,payoff,vols
177,20201201,20201218,C,2900000,758.1,764.2,E,761.15,2900.0,1,4.672338e-14
178,20201201,20201218,C,2905000,753.0,759.2,E,756.10,2905.0,1,9.840124e-13
179,20201201,20201218,C,2910000,748.0,754.2,E,751.10,2910.0,1,9.558152e-13
180,20201201,20201218,C,2915000,743.0,751.4,E,747.20,2915.0,1,1.857993e-13
181,20201201,20201218,C,2920000,738.0,744.3,E,741.15,2920.0,1,9.807671e-13
182,20201201,20201218,C,2925000,733.2,739.3,E,736.25,2925.0,1,5.600331e-14
183,20201201,20201218,C,2930000,728.1,734.3,E,731.20,2930.0,1,2.338384e-15
184,20201201,20201218,C,2935000,723.1,729.3,E,726.20,2935.0,1,9.764917e-13
185,20201201,20201218,C,2940000,718.1,724.4,E,721.25,2940.0,1,1.377754e-14
186,20201201,20201218,C,2945000,713.1,719.4,E,716.25,2945.0,1,9.993877e-13


In [31]:
brentq(lambda x: 661.7 -
                                BlackScholesLognormalCall(3662.45, 3000, 0.14/100.0, x, days_to_expiry/365),
                                -1e-12, 10.0)

4.513005756339833e-13

In [30]:
brentq(lambda x: 657.7 -
                                BlackScholesLognormalCall(3662.45, 3005, 0.14/100.0, x, days_to_expiry/365),
                                1e-12, 10.0)

0.29474056232626716

In [8]:
beta = 0.7      # set Beta value

def sabrcalibration(x, strikes, vols, F, T) -> float:
    """
    Function to obtain the error between estimated(using parameters in x) and actual given(vols) sigmas
    float
    
    """
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR(F, strikes[i], T,
                           x[0], beta, x[1], x[2]))**2
    
    return err

def impliedVolatility(S, K, r, price, T, payoff) -> float:
    """
    Function to calculate implied vol using blackscholes
    """

In [9]:
def SABR(F, K, T, alpha, beta, rho, nu): 
    """
    Function for SABR sigma calculation based on given parameters
    
    """
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma
